In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

2024-11-18 16:52:11.500697: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 16:52:12.001926: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 16:52:12.348289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731970332.603520   10437 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731970332.672988   10437 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 16:52:13.609906: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
# Configuración del directorio y parámetros de las imágenes
data_dir = './dataset'  # Ruta al dataset que tienes en tu PC
image_size = (224, 224)  # Tamaño de las imágenes que utilizará el modelo
batch_size = 32  # Tamaño del lote para entrenamiento
num_classes = 12  # Número de clases

In [3]:
# Generador de datos con separación de entrenamiento y validación
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalización y división

# Conjunto de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multi-clase para reconocer tipo de fruta y frescura
    subset='training'
)

# Conjunto de validación
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 11759 images belonging to 12 classes.
Found 2934 images belonging to 12 classes.


In [4]:
# Crear lista de nombres de clases basados en el orden de los índices
class_names = list(train_generator.class_indices.keys())
print(class_names)


['fresh_apple', 'fresh_banana', 'fresh_bitter_gourd', 'fresh_capsicum', 'fresh_orange', 'fresh_tomato', 'stale_apple', 'stale_banana', 'stale_bitter_gourd', 'stale_capsicum', 'stale_orange', 'stale_tomato']


In [5]:
# Definición del modelo de red neuronal convolucional (CNN)
model = Sequential([
    # Primera capa convolucional con regularización L2
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    # Segunda capa convolucional con regularización L2
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    
    # Tercera capa convolucional con regularización L2
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    
    # Aplanamiento para conectar con las capas densas
    Flatten(),
    
    # Primera capa densa con regularización L2 y Dropout
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),  # Aplica Dropout para reducir overfitting
    
    # Capa de salida para clasificar (softmax para multi-clase)
    Dense(num_classes, activation='softmax')  # 'num_classes' es el número total de categorías
])

/home/croubs/Escritorio/Repos/PIA-IA-2.5/env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-18 16:52:17.391546: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2024-11-18 16:52:17.454065: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2024-11-18 16:52:17.480393: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2024-11-18 16:52:17.572638: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free s

In [6]:
# Compilación del modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [8]:
# Entrenamiento del modelo con más epochs
epochs = 30  # Aumentado
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[lr_reduction, early_stop]  # Agrega los callbacks aquí
)

/home/croubs/Escritorio/Repos/PIA-IA-2.5/env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


2024-11-18 16:52:19.220551: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2024-11-18 16:52:19.861476: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.


368/368 ━━━━━━━━━━━━━━━━━━━━ 826s 2s/step - accuracy: 0.4365 - loss: 2.7525 - val_accuracy: 0.6738 - val_loss: 1.3337 - learning_rate: 0.0010
Epoch 2/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 771s 2s/step - accuracy: 0.7270 - loss: 1.2789 - val_accuracy: 0.7369 - val_loss: 1.1088 - learning_rate: 0.0010
Epoch 3/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 788s 2s/step - accuracy: 0.7465 - loss: 1.1324 - val_accuracy: 0.7621 - val_loss: 0.9883 - learning_rate: 0.0010
Epoch 4/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 738s 2s/step - accuracy: 0.7759 - loss: 1.0277 - val_accuracy: 0.8003 - val_loss: 0.8831 - learning_rate: 0.0010
Epoch 5/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 757s 2s/step - accuracy: 0.7843 - loss: 0.9806 - val_accuracy: 0.8183 - val_loss: 0.8986 - learning_rate: 0.0010
Epoch 6/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 755s 2s/step - accuracy: 0.8022 - loss: 0.9437 - val_accuracy: 0.7829 - val_loss: 0.8776 - learning_rate: 0.0010
Epoch 7/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 725s 2s/step - accuracy: 0.7948 - loss: 0.9448 - val_

In [9]:
# Evaluación del modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Loss en validación: {val_loss}")
print(f"Precisión en validación: {val_accuracy}")

92/92 ━━━━━━━━━━━━━━━━━━━━ 43s 469ms/step - accuracy: 0.9032 - loss: 0.4760
Loss en validación: 0.4803222715854645
Precisión en validación: 0.9049080014228821


In [10]:
# Guardar el modelo entrenado
model.save("modelo_clasificacion_frutas.h5")